In [1]:
from collections import deque, defaultdict

In [9]:
def read_input(filename):
    with open(filename, 'r') as f:
        grid = {}
        units = []
        for i, line in enumerate(f.readlines()):
            for j, c in enumerate(line.strip()):
                grid[i, j] = c
                if c in ['G', 'E']:
                    units.append([c, [i, j], 200])
    return grid, units

def shortest_path_part(grid, start, end):
    q = deque([start])
    explored = set([start])
    prev = {}
    found = False
    while q:
        v = q.popleft()
        if v == end:
            found = True
            break
        for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            i = v[0] + di
            j = v[1] + dj
            if (i, j) in grid and grid[i, j] == '.':
                if (i, j) not in explored:
                    explored.add((i, j))
                    prev[(i, j)] = v
                    q.append((i, j))
    if found:
        node = end
        path = [end]
        while node != start:
            path.append(prev[node])
            node = prev[node]
        path.reverse()
        return True, path
    else:
        return False, None
    
def shortest_path(grid, start, end):
    shortest_len = 1e6
    shortest = None
    for di, dj in [(-1, 0), (0, -1), (0, 1), (1, 0)]:
        i, j = start[0] + di, start[1] + dj
        if (i, j) in grid and grid[i, j] == '.':
            found, path = shortest_path_part(grid, (i, j), end)
            if found:
                if len(path) < shortest_len:
                    shortest = path
                    shortest_len = len(path)
    return shortest

def get_range(grid, target_pos):
    i, j = target_pos
    target_range = []
    for di, dj in [(-1, 0), (0, -1), (0, 1), (1, 0)]:
        if grid.get((i + di, j + dj), '#') == '.':
            target_range.append((i + di, j + dj))
    return target_range

def turn(unit, units, grid, elf_power):
    did_something = False
    unit_type, (i, j), _ = unit
    target_type = 'E' if unit_type == 'G' else 'G'
    in_range = False
    for target in sorted([u for u in units if u[0] == target_type], key=lambda x: [x[2], x[1][0], x[1][1]]):
        i_t, j_t = target[1]
        if (i, j) in [(i_t + di, j_t + dj) for (di, dj) in [(-1, 0), (1, 0), (0, -1), (0, 1)]]:
            in_range = True
            target[2] -= elf_power if unit_type == 'E' else 3
            did_something = True
            break
    units_new = []
    for u in units:
        if u[2] <= 0:
            if u[0] == 'E':
                return None, None, None
            grid[u[1][0], u[1][1]] = '.'
        else:
            units_new.append(u)
    units = units_new
    if not in_range:
        shortest = None
        shortest_len = 1e6
        for target in sorted([u for u in units if u[0] == target_type], key=lambda x: x[1]):
            for range in get_range(grid, target[1]):
                path = shortest_path(grid, (i, j), range)
                if path is not None and len(path) < shortest_len:
                    shortest = path
                    shortest_len = len(path)
        if shortest is not None:
            i, j = unit[1]
            grid[i, j] = '.'
            i, j = shortest[0]
            unit[1] = [i, j]
            grid[i, j] = unit[0]
            did_something = True
            for target in sorted([u for u in units if u[0] == target_type], key=lambda x: [x[2], x[1][0], x[1][1]]):
                i_t, j_t = target[1]
                if (i, j) in [(i_t + di, j_t + dj) for (di, dj) in [(-1, 0), (1, 0), (0, -1), (0, 1)]]:
                    in_range = True
                    target[2] -= elf_power if unit_type == 'E' else 3
                    break
            units_new = []
            for u in units:
                if u[2] <= 0:
                    if u[0] == 'E':
                        return None, None, None
                    grid[u[1][0], u[1][1]] = '.'
                else:
                    units_new.append(u)
            units = units_new
    return did_something, unit, units

def round(grid, units, part2=False, elf_power=3):
    something_happend = False
    for unit in sorted(units, key=lambda x: x[1]):
        if unit[2] <= 0:
            continue
        did_something, unit, units = turn(unit, units, grid, elf_power)
        if part2 and (did_something is None) and (unit is None) and (units is None):
            return None, None, None
        something_happend = something_happend or did_something
        elf_left = False
        goblin_left = False
        for u in units:
            if u[0] == 'E':
                elf_left = True
            if u[0] == 'G':
                goblin_left = True
        if not (elf_left and goblin_left):
            something_happend = False
            break
    return something_happend, grid, units

def print_state(grid, units):
    i_max = max(grid.keys(), key=lambda x: x[0])[0]
    j_max = max(grid.keys(), key=lambda x: x[1])[1]
    for i in range(i_max + 1):
        gp = ''.join(grid[i, j] for j in range(j_max + 1))
        up = []
        for u in sorted(units, key=lambda x: x[1][1]):
            if u[1][0] == i:
                up.append(f'{u[0]}({u[2]})')
        print(gp + '   ' + ', '.join(up))

In [5]:
grid, units = read_input('15_input.txt')
print_state(grid, units)
n_rounds = 0
while True:
    something_happend, grid, units = round(grid, units)
    n_rounds += 1
    if not something_happend:
        break
print_state(grid, units)
print(f'Rounds: {n_rounds - 1}')
sum([u[2] for u in units]) * (n_rounds - 1)

################################   
###############.##...###########   
##############..#...G.#..#######   G(200)
##############.............#####   
###############....G....G......#   G(200), G(200)
##########..........#..........#   
##########................##..##   
######...##..G...G.......####..#   G(200), G(200)
####..G..#G...............####.#   G(200), G(200)
#######......G....G.....G#####E#   G(200), G(200), G(200), E(200)
#######.................E.######   E(200)
########..G...............######   G(200)
######....G...#####E...G....####   G(200), E(200), G(200)
######..G..G.#######........####   G(200), G(200)
###.........#########.......E.##   E(200)
###..#..#...#########...E.....##   E(200)
######......#########.......####   
#####...G...#########.....######   G(200)
#####G......#########.....######   G(200)
#...#G..G....#######......######   G(200), G(200)
###...##......#####.......######   
####..##..G........E...E..######   G(200), E(200), E(200)
#####.####.....######.

229798

In [19]:
# 50 -> 25 -> 13 -> 19 -> 16 -> 17 -> 18 -> 19
grid, units = read_input('15_input.txt')
#print_state(grid, units)
n_rounds = 0
while True:
    something_happend, grid, units = round(grid, units, part2=True, elf_power=19)
    if (something_happend is None) and (grid is None) and (units is None):
        print(f'Fail at round {n_rounds}!')
        break
    n_rounds += 1
    if not something_happend:
        break
print_state(grid, units)
print(f'Rounds: {n_rounds - 1}')
sum([u[2] for u in units]) * (n_rounds - 1)

################################   
###############.##...###########   
##############..#.....#..#######   
##############.............#####   
###############.......E.E......#   E(200), E(56)
##########..........#..E.......#   E(113)
##########..........E.....##..##   E(200)
######...##.........E....####..#   E(86)
####.....#........EE.E....####.#   E(194), E(8), E(200)
#######..................#####.#   
#######...................######   
########..................######   
######.....E..#####.........####   E(137)
######.......#######.E......####   E(200)
###.........#########.........##   
###..#..#...#########.........##   
######......#########.......####   
#####.......#########.....######   
#####.......#########.....######   
#...#........#######......######   
###...##......#####.......######   
####..##..................######   
#####.####.....######...########   
###########..#...####.....######   
###############...####..#...####   
###############...###...#...####   
##

52972